In [8]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.models.phrases import Phraser, Phrases

In [5]:
ls data/

Invalid switch - "".


In [7]:
df = pd.read_csv('data/job_ofer.csv')
df

,title,company_name,address,description,seniority_level,employment_type,job_function,industries
0,Machine Learning Engineer,Intellipro Group Inc,"Palo Alto, CA, US","['About The Company', ""W*** is reshaping the f...",Entry level,Full-time,Engineering,Information Technology and Services
1,Deep Learning Applied Researcher - Chicago,Ethosia,"Chicago, IL, US","['תיאור המשרה', 'Deep learning for Computer Vi...",Associate,Full-time,Other,Information Technology and Services
2,Machine Learning Engineer,Motorola Solutions,"Chicago, IL, US","['Company Overview', 'At Motorola Solutions, w...",Entry level,Full-time,Engineering,Information Technology and Services
3,Machine Learning / Data Scientist,Proprius LLC,"San Francisco, CA, US",['Our client is a digital invention agency foc...,Entry level,Full-time,Engineering,Information Technology and Services
4,Cloud Architect,TCS,"Framingham, Massachusetts, United States","['Technical/Functional Skills', ' ', 'Good to ...",Mid-Senior level,Full-time,Engineering,Information Technology and Services
5,Data Scientist,Nationwide,"Columbus, OH, US",['If you’re passionate about becoming a Nation...,Entry level,Full-time,Engineering,Information Technology and Services
6,Store Room Clerk,Nestlé,"Dallas, TX, US",['As the nation’s leading bottled water compan...,Not Applicable,Full-time,Production,Consumer Goods
7,"Director of Product, US",Monzo Bank,"Los Angeles, CA, US","['Location:', ' Los Angeles', 'Monzo is a bank...",Director,Full-time,Product Management,Computer Software
8,"Recruiting Manager, AD-0301-00-Census-EXT-GB",Department of Commerce,"Winston-Salem, NC, US","['Duties', 'Summary', 'ATTENTION: The employee...",Associate,Part-time,Human Resources,Nonprofit Organization Management
9,4710 - Bilingual Engineer - German (Germany or...,Qualitest,"Austin, TX, US","['Description', 'Are you interested in working...",Entry level,Full-time,Engineering,Information Technology and Services


In [9]:
title_corpus = df['title'].map(simple_preprocess)

In [59]:
type(title_corpus)

pandas.core.series.Series

In [23]:
title_tagged = [TaggedDocument(words=sent, tags=[i]) for i, sent in enumerate(title_corpus)]

In [32]:
title_model = Doc2Vec(vector_size=300, window_size=5, min_count=1)

In [33]:
title_model.build_vocab(title_tagged)

In [34]:
title_model.train(title_tagged, total_examples = title_model.corpus_count, epochs=10)

In [35]:
title_model.docvecs.most_similar(0)

[(6147, 0.8873828649520874),
 (9895, 0.8681989908218384),
 (33091, 0.8406906723976135),
 (370, 0.8352864384651184),
 (318, 0.801518440246582),
 (36082, 0.7719108462333679),
 (9677, 0.7678601145744324),
 (4293, 0.7676063776016235),
 (33522, 0.7593362927436829),
 (26393, 0.7589328289031982)]

In [28]:
df[df.index==0].title

0    Machine Learning Engineer
Name: title, dtype: object

In [36]:
df[df.index==6147].title

6147    Senior Machine Learning Engineer (Relocate to ...
Name: title, dtype: object

Adding connected words eg. machine learning = machine_learning

In [67]:
title_corpus = df['title'].map(simple_preprocess)
title_bigram = Phraser(Phrases(title_corpus, min_count=1, threshold=1))
title_bigram[simple_preprocess('Machine Learning Applied Researcher - Chicago')]

def prepare_corpus(corpus, bigram):
    for sent in corpus:
        yield bigram[sent]+sent
        
ext_corpus = list(prepare_corpus(title_corpus, title_bigram))
ext_corpus = pd.Series(ext_corpus)
ext_corpus[0]
# model2 = Word2Vec(ext_corpus, size=100, window=2, min_count=1)

['machine_learning', 'engineer', 'machine', 'learning', 'engineer']

In [75]:
title_tagged = [TaggedDocument(words=sent, tags=[i]) for i, sent in enumerate(ext_corpus)]
title_model = Doc2Vec(vector_size=300, window_size=7, min_count=1)
title_model.build_vocab(title_tagged)
title_model.train(title_tagged, total_examples = title_model.corpus_count, epochs=10)

In [76]:
title_model.docvecs.most_similar(0)

[(34619, 0.859139621257782),
 (446, 0.8547508716583252),
 (3223, 0.8528279066085815),
 (12266, 0.8499792814254761),
 (417, 0.846861720085144),
 (7139, 0.8463007807731628),
 (3130, 0.8190166354179382),
 (11876, 0.8128637671470642),
 (26393, 0.7958504557609558),
 (30054, 0.7947952151298523)]

In [78]:
df[df.index==34619].title

34619    CNC Engineer
Name: title, dtype: object

In [79]:
df[df.index==3223].title

3223    Kibana - Visualisations Engineer
Name: title, dtype: object

Model based on description of job

In [82]:
desc =  df['description'].map(simple_preprocess)
desc_tagged = [TaggedDocument(words=sent, tags=[i]) for i, sent in enumerate(desc)]
desc_model = Doc2Vec(vector_size=300, window_size=5, min_count=1)
desc_model.build_vocab(desc_tagged)
desc_model.train(desc_tagged, total_examples = desc_model.corpus_count, epochs=10)
desc_model.docvecs.most_similar(0)

[(26379, 0.43502485752105713),
 (19853, 0.42695116996765137),
 (26386, 0.41992154717445374),
 (343, 0.4186694324016571),
 (8527, 0.4144522547721863),
 (24391, 0.41300028562545776),
 (6542, 0.4110393226146698),
 (29956, 0.4097056984901428),
 (27769, 0.40685296058654785),
 (24703, 0.40584301948547363)]

In [86]:
df[df.index==0].description.values

array(['[\'About The Company\', "W*** is reshaping the future of delivery. We are an on-demand drone delivery service that can deliver food, medicine or other items within minutes. We\'ve also developed an unmanned traffic management platform to safely route drones through the sky. Our service is faster, safer and produces far less pollution than traditional delivery.", \'About The Role\', \'As a Machine Learning Engineer you will help develop models to support the next generation of intelligence that backs our flight planning and navigation solutions. In this way, you will play an important part in our larger goal of building a state-of-the-art delivery system that safely flies thousands of autonomous aircraft every day around people, buildings, and terrain, with or without GPS, day or night, rain or shine. Developing machine-learning-based models with the real world application in mind is a difficult problem, but working at the intersection of R&D and production is a unique and exhil

In [85]:
df[df.index==26379].description.values

array(["['We’re changing the way people think about transportation. Not that long ago we were just an app to request premium black cars in a few metropolitan areas. Now we’re a part of the logistical fabric of more than 700 cities around the world. Whether it’s a ride, a sandwich, or a package, we use technology to give people what they want, when they want it.', 'For the people who drive with Uber, our app represents a flexible new way to earn money. For cities, we help strengthen local economies, improve access to transportation, and make streets safer.', 'And that’s just what we’re doing today. We’re thinking about the future, too. With teams working on new modalities, self-driving cars and even urban air transportation, we’re in for the long haul. We’re reimagining how people and things move from one place to the next.', 'About The Role', 'At Uber, we’re building an advanced simulation system - a key component to the development of our autonomous vehicle technology. Come join our g